In [12]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [0]:
train_dir = "/content/drive/My Drive/Datasets/Men & Women Classification/Train"
validation_dir = "/content/drive/My Drive/Datasets/Men & Women Classification/Validation"


In [0]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [0]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        
        if i * batch_size >= sample_count:
            break
        return features, labels
    

In [42]:
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 600)

Found 1992 images belonging to 2 classes.
Found 596 images belonging to 2 classes.


In [0]:
from tensorflow.keras import models
from tensorflow.keras import layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


In [44]:
len(model.trainable_weights)

30

In [0]:
conv_base.trainable = False

In [46]:
len(model.trainable_weights)

4

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [49]:
train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(150, 150),
                    batch_size=20,
                    class_mode='binary'
)

Found 1992 images belonging to 2 classes.


In [50]:
validation_generator = test_datagen.flow_from_directory(
                        validation_dir,
                        target_size=(150, 150),
                        batch_size=20,
                        class_mode='binary'
)

Found 596 images belonging to 2 classes.


In [0]:
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=2e-5),
metrics=['acc'])

In [52]:
history = model.fit_generator(
                                train_generator,
                                steps_per_epoch=50,
                                epochs=30,
                                validation_data=validation_generator,
                                validation_steps=50
)

Epoch 1/30
50/50 [==============================] - 48s 960ms/step - loss: 0.7226 - acc: 0.5460 - val_loss: 0.6279 - val_acc: 0.6486
Epoch 2/30
50/50 [==============================] - 48s 953ms/step - loss: 0.7005 - acc: 0.5786 - val_loss: 0.6037 - val_acc: 0.6946
Epoch 3/30
50/50 [==============================] - 47s 941ms/step - loss: 0.6565 - acc: 0.6200 - val_loss: 0.5739 - val_acc: 0.6784
Epoch 4/30
50/50 [==============================] - 46s 926ms/step - loss: 0.6321 - acc: 0.6462 - val_loss: 0.5629 - val_acc: 0.7048
Epoch 5/30
50/50 [==============================] - 46s 925ms/step - loss: 0.6218 - acc: 0.6590 - val_loss: 0.5453 - val_acc: 0.7228
Epoch 6/30
50/50 [==============================] - 48s 956ms/step - loss: 0.5908 - acc: 0.6855 - val_loss: 0.5388 - val_acc: 0.7399
Epoch 7/30
50/50 [==============================] - 47s 932ms/step - loss: 0.6048 - acc: 0.6540 - val_loss: 0.5186 - val_acc: 0.7530
Epoch 8/30
50/50 [==============================] - 48s 953ms/step - 

In [0]:
model.save("Men&WomenClassification.h5")

FINE TUNING

In [0]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [0]:
model.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=1e-5),
                metrics=['acc']
            )

In [62]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=100,
                epochs=10,
                validation_data=validation_generator,
                validation_steps=50)

Epoch 1/10
100/100 [==============================] - 74s 742ms/step - loss: 0.3162 - acc: 0.8609 - val_loss: 0.3871 - val_acc: 0.8454
Epoch 2/10
100/100 [==============================] - 73s 729ms/step - loss: 0.2897 - acc: 0.8730 - val_loss: 0.3670 - val_acc: 0.8599
Epoch 3/10
100/100 [==============================] - 74s 743ms/step - loss: 0.2996 - acc: 0.8660 - val_loss: 0.3619 - val_acc: 0.8599
Epoch 4/10
100/100 [==============================] - 74s 741ms/step - loss: 0.2731 - acc: 0.8775 - val_loss: 0.4007 - val_acc: 0.8524
Epoch 5/10
100/100 [==============================] - 74s 736ms/step - loss: 0.2625 - acc: 0.8855 - val_loss: 0.4170 - val_acc: 0.8337
Epoch 6/10
100/100 [==============================] - 74s 735ms/step - loss: 0.2616 - acc: 0.8926 - val_loss: 0.3488 - val_acc: 0.8589
Epoch 7/10
100/100 [==============================] - 75s 750ms/step - loss: 0.2536 - acc: 0.9016 - val_loss: 0.3583 - val_acc: 0.8574
Epoch 8/10
100/100 [==============================] - 7

In [1]:
model.save("Men&WomenClassification.h5")

NameError: ignored